
Targets: Above 98% test_acc with very less number of params  
Results:  
    best test_acc = 99% on 15th epoch  

    total parameters = 7434  
Analysis:  It has Basic skeleton of Lighter model with Batch normalization  
File Link:  

# 1. Import external libraries

In [21]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
%matplotlib inline
import matplotlib.pyplot as plt


#### Basically working model
#### Img Aug RandomRotation
#### StepLR Scheduler
#### Train & Test Graphs

# 2. Convolutional Neural Network (model) architecture

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        chan = 8
        adv = 16
        self.conv1 = nn.Conv2d(1, chan, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2 = nn.Conv2d(chan, chan, 3)
        self.bn2 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(chan, chan, 3)
        self.bn3 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv4 = nn.Conv2d(chan, chan, 3)
        self.bn4 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(chan, chan, 3)
        self.bn5 = nn.BatchNorm2d(num_features=chan, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv6 = nn.Conv2d(chan, adv, 3)
        self.bn6 = nn.BatchNorm2d(num_features=adv, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv7 = nn.Conv2d(adv, adv, 3)
        self.conv8 = nn.Conv2d(adv, 10, 3)

    def forward(self, x):
        x = self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))
        x = self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x))))))
        x = (F.relu(self.conv6(self.bn5(F.relu(self.conv5(x))))))
        x = self.conv8(self.conv7(x))
        #print(x.shape)
        x = x.view(-1, 10) # Raja ToDo Try printing shape here
        return F.log_softmax(x)

# 3. Display summary of model

In [23]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available() #bool
str_gpu_cpu = "cuda" if use_cuda else "cpu" #string
device = torch.device(str_gpu_cpu)
print("device is " + str_gpu_cpu)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

device is cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 26, 26]             584
       BatchNorm2d-4            [-1, 8, 26, 26]              16
         MaxPool2d-5            [-1, 8, 13, 13]               0
            Conv2d-6            [-1, 8, 11, 11]             584
       BatchNorm2d-7            [-1, 8, 11, 11]              16
            Conv2d-8              [-1, 8, 9, 9]             584
       BatchNorm2d-9              [-1, 8, 9, 9]              16
           Conv2d-10              [-1, 8, 7, 7]             584
      BatchNorm2d-11              [-1, 8, 7, 7]              16
           Conv2d-12             [-1, 16, 5, 5]           1,168
           Conv2d-13             [-1, 16, 3, 3]           2,320
           Conv2d-14    

<ipython-input-22-96cfc22237e9>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


# 4. Preparation of dataset  

In [24]:

SEED = 1
torch.manual_seed(SEED)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


# 5. Train and Test Functions

In [25]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_losses.append(loss)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            test_losses.append(test_loss)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\n  Test set: Average loss: {:.4f}, Test Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc.append(100. * correct / len(test_loader.dataset))


# 6. Run the model with a device and an optimizer

In [26]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21): #RAJA changed epochs
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

#With params = 3846, on 20th epoch, test accuracy = 98.98%
#With params = 5146, on 11th epoch, test accuracy = 99.20% [reduced afterwards]

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-22-96cfc22237e9>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
epoch=1 loss=0.06281910091638565 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.29it/s]



  Test set: Average loss: 0.0851, Test Accuracy: 9746/10000 (97.46%)



epoch=2 loss=0.08258814364671707 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.05it/s]



  Test set: Average loss: 0.0626, Test Accuracy: 9808/10000 (98.08%)



epoch=3 loss=0.08882034569978714 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.51it/s]


...


  Test set: Average loss: 0.0345, Test Accuracy: 9902/10000 (99.02%)



epoch=19 loss=0.01065776590257883 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.08it/s]



  Test set: Average loss: 0.0315, Test Accuracy: 9901/10000 (99.01%)



epoch=20 loss=0.03852921351790428 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.15it/s]



  Test set: Average loss: 0.0375, Test Accuracy: 9891/10000 (98.91%)



Raja ToDo :
Try below ::
1. BatchNormalization
2. Dropout
3. LR scheduler
4. GAP